First, I'll reproduce the code from Jeremy Howard's notebook:
https://www.kaggle.com/code/jhoward/how-does-a-neural-net-really-work

In [20]:
from matplotlib import pyplot
import numpy
import torch
from ipywidgets import interact
from fastai.basics import *

plt.rc('figure', dpi=90)

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylim=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylim: plt.ylim(ylim)
    plt.plot(x, f(x), color)
    if title is not None: plt.title(title)


def f(x): return 3*x**2 + 2*x + 1
def quad(a, b, c, x): return a * x ** 2 + b * x + c
def make_quad(a, b, c): return partial(quad, a, b, c)
def mean_absolute_error(predictions, actuals): return torch.abs(predictions - actuals).mean()
def noise(x, scale): return numpy.random.normal(scale=scale, size=x.shape)
def add_noise(x, mult, add): return x * (1 + noise(x, mult)) + noise(x, add)
numpy.random.seed(42)
x = torch.linspace(-2, 2, steps=20)[:,None]
y = add_noise(f(x), 0.15, 1.5)
def quad_mean_absolute_error(params):
	f = make_quad(*params)
	return mean_absolute_error(f(x), y)


In [21]:
abc = torch.tensor([1.1, 1.1, 1.1])
abc.requires_grad_()

loss = quad_mean_absolute_error(abc)
loss

tensor(2.4219, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [22]:
loss.backward()

In [23]:
abc.grad

tensor([-1.3529, -0.0316, -0.5000])

In [24]:
with torch.no_grad():
    abc -= abc.grad*0.01
    loss = quad_mean_absolute_error(abc)
    
print(f'loss={loss:.2f}')

loss=2.40


In [25]:
for i in range(10):
    loss = quad_mean_absolute_error(abc)
    loss.backward()
    with torch.no_grad(): abc -= abc.grad*0.01
    print(f'Step={i}; loss={loss:.2f}')

Step=0; loss=2.40
Step=1; loss=2.36
Step=2; loss=2.30
Step=3; loss=2.21
Step=4; loss=2.11
Step=5; loss=1.98
Step=6; loss=1.85
Step=7; loss=1.72
Step=8; loss=1.58
Step=9; loss=1.46


In [26]:
def rectified_linear(m,b,x):
    y = m*x+b
    return torch.clip(y, 0.)

In [27]:
@interact(m=1.5,b=1.5)
def plot_relu(m,b):
    plot_function(partial(rectified_linear, m,b))

interactive(children=(FloatSlider(value=1.5, description='m', max=4.5, min=-1.5), FloatSlider(value=1.5, descr…

### Now let's try and "manually" (meaning: without a predefined nn architecture) create a super-basic NN and optimize it for the Titanic Competition
https://www.kaggle.com/competitions/titanic

In [47]:
import pandas as pd

# import csv
csv = pd.read_csv("data/titanic/train.csv")

# cleanup superfluous data and convert columns to usable numbers
dataframe = pd.DataFrame(csv)
dataframe = dataframe.drop(columns=["PassengerId","Ticket", "Name"])
dataframe['isMale'] = dataframe['Sex'].apply(lambda gender : 1 if gender == 'male' else 0)
dataframe['isEmbarkedS']

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,isMale
0,0,3,male,22.0,1,0,7.2500,NaN,S,1
1,1,1,female,38.0,1,0,71.2833,C85,C,0
2,1,3,female,26.0,0,0,7.9250,NaN,S,0
3,1,1,female,35.0,1,0,53.1000,C123,S,0
4,0,3,male,35.0,0,0,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S,1
887,1,1,female,19.0,0,0,30.0000,B42,S,0
888,0,3,female,NaN,1,2,23.4500,NaN,S,0
889,1,1,male,26.0,0,0,30.0000,C148,C,1
